# IT Company Project — Documentation & Development Notebook

This notebook documents the `it-Ideatask` project, setup steps for running Java/Gradle/JUnit/Mockito, SonarQube integration notes, example code snippets from the project, and simple unit-test examples (Mockito where relevant).

**Environment**
- OS: Windows (confirmed)
- Project root: `C:\Users\danie\Documents\Solvd\it-Ideatask`
- JDK: 21
- Recommended: IntelliJ IDEA (already used), Jupyter (for viewing this notebook)


Prompt Engineering Augmentation

Project: Automating Testing with JUnit Using the AI Fluency Framework (Anthropic, V1.5)
Alejandro Ramirez Herrera

1.	Overview

This project applies the AI Fluency Framework’s 4Ds—Delegation, Description, Discernment, and Diligence—to the automation of JUnit 5 test creation using AI. The goal is to reduce manual testing time while maintaining high-quality, comprehensive test coverage for Java classes. The outcome includes generated JUnit test code, documentation, and a reusable prompt workflow.
2. Delegation

Delegation involves selecting the right AI tools and defining clear roles for human–AI collaboration. In this case, the AI (ChatGPT, GPT-5) is responsible for generating, refactoring, and documenting JUnit tests, while the human developer provides source code, validates outputs, and integrates the generated tests into the project.
Delegation Plan:
Role	Responsibility
Human	Provide source code, review and validate AI-generated tests.
AI	Generate JUnit tests, refactor code, suggest edge cases, document results.
Prompt Engineering Technique: Role prompting + context-rich input.
Prompt:
“You are a senior Java QA engineer. Given this Java class, generate comprehensive JUnit 5 test cases using Mockito where appropriate.”
3. Description

Description focuses on communicating tasks effectively to prompt useful AI behaviors. It includes product, process, and performance description techniques.
Prompt Examples:
1. Product-focused prompt:
“Generate a complete UserServiceTest class for the following Java code. Use JUnit 5 and Mockito. Include tests for valid, invalid, and boundary cases.”
2. Process prompt:
“Review the tests you just created. Identify missing branches, redundant tests, and untested exception paths. Regenerate only the improved sections.”
3. Performance prompt:
“Create a reusable JUnit test generation template prompt that can be applied to any Java class with methods.”
4. Discernment

Discernment refers to the ability to assess AI outputs for quality, relevance, and completeness. Evaluation is based on test coverage, correctness, and clarity.
Evaluation Metrics:
Metric	Target
Test coverage	≥ 80%
Compilation errors	None
Assertion quality	All tests include meaningful asserts
Time saved	≥ 50% over manual test writing
Example prompt for critique:
“Assess the following JUnit tests for clarity, naming convention, and edge case coverage. Suggest 3 specific improvements.”
5. Diligence

Diligence ensures accountability and ethical use of AI in the development process. This includes verifying originality, disclosing AI involvement, and validating results.
Transparency Statement:
“JUnit test cases were co-generated using ChatGPT (GPT-5). The developer reviewed and validated all code before integration.”
// Co-generated with AI assistance (GPT-5), reviewed and validated by Alejandro Ramirez
Before deployment, all AI-generated tests must be verified locally to ensure they run correctly and do not introduce false positives or errors.
6. Mapping the Framework to Prompt Engineering

AI Fluency Competency	Prompt Technique	Purpose
Delegation	Role prompting, task breakdown	Define roles, clarify objectives
Description	Context-rich, iterative, and format-controlled prompts	Guide AI behavior for reliable outputs
Discernment	Critique prompting, self-reflection prompts	Evaluate and refine AI outputs
Diligence	Transparency prompts, validation requests	Ensure accountability and ethical use
7. Refined Final Prompt Example

“Act as a senior QA automation engineer. For the following Java class, create high-quality JUnit 5 test cases that:
- Cover valid, invalid, and boundary conditions
- Use Mockito for dependencies
- Follow the naming convention method_scenario_expectedResult()
- Include meaningful assertions and avoid redundancy
- Return only valid, compilable Java code.”


Prompt for Test Generation/Improvement

Scalability	

"Write a JUnit test class, LoadTestSimulation, for the CostEstimator service. This test must use a parameterized or repeated test approach to simulate 1,000 concurrent requests for cost calculation, ensuring the method completes within a 5-second timeout for the entire batch. Focus on thread-safety if concurrent services are involved."
Security	"Develop a JUnit test method, testSanitizeUserInput, within the MainTest class. This test should verify that all methods accepting user input (e.g., project name, comments, etc.) correctly sanitize input strings by checking for common injection vectors like <script>, DROP TABLE, or excessive character lengths, ensuring that security-critical methods are never called with unsanitized data."

Maintainability/Clarity	

"Refactor and expand the existing tests in ProjectTest. Ensure every test method adheres to the Arrange-Act-Assert (AAA) pattern, uses clear and descriptive names (e.g., testCalculateDiscount_HighComplexity_Applies10Percent), and includes mocking for external dependencies (like the BudgetService or CostEstimator) to achieve 100% test isolation."

Performance	

"Create a dedicated JUnit test, ReportGenerationPerformanceTest, to measure the execution time of the generateProjectReport method. Use the Timeout annotation or a manual measurement to assert that report generation for a large, complex project (e.g., 50 features, 10 developers) completes in under 500 milliseconds on a standard build machine."

Reliability/Edge Cases	

"Write a comprehensive set of JUnit tests for the Team class and related manager logic. Include tests for all edge cases observed in the logs: invalid negative team size, non-numeric inputs for hourly rates, invalid manager level strings (WRONG ANSWER cases), and ensure that resource allocation or budget checks handle zero or null values gracefully without throwing uncaught exceptions."


----
## 1) Quick setup checklist

1. Ensure JDK 21 is installed and `java`/`javac` are on PATH.
2. Install Gradle wrapper or use the project's `gradlew` (Windows: `.\gradlew`).
3. Run tests from the project root with Gradle:
   ```powershell
   .\gradlew test
   ```
4. If you want Java support in Jupyter, consider installing IJava (instructions below).
5. SonarQube: have a running SonarQube server (local or remote) and `sonar-scanner` installed or configure scanner in Gradle/Maven.


----
## 2) IJava kernel (Jupyter Java kernel) 

Cloned `SpencerPark/IJava`. Typical install steps on Windows:
1. Build the project with Gradle (from the IJava directory):
   ```powershell
   .\gradlew install
   ```
2. If `build.gradle` or `buildSrc` uses old Gradle features, you may need Gradle wrapper to pick compatible version. Use `--refresh-dependencies` if needed.
3. After successful build, a kernel spec is installed and `jupyter kernelspec list` should include `java`.

If building `IJava` fails with Gradle plugin errors (e.g. `compile()` method not found in modern Gradle), try opening `buildSrc/build.gradle` and change `compile` to `implementation` or use a Gradle version compatible with the project. Be careful — modifying the kernel build scripts may be needed.

Alternatively, consider using VS Code with Java extensions or IntelliJ's built-in notebook features instead of a Java kernel in Jupyter.

----
## 3) SonarQube integration (high-level)

Add SonarQube scanning to Gradle (example `build.gradle` snippet):
```groovy
plugins {
  id "org.sonarqube" version "4.3.0.3225"
}
sonarqube {
  properties {
    property "sonar.projectKey", "your-key"
    property "sonar.host.url", "http://localhost:9000"
    property "sonar.login", "YOUR_TOKEN"
  }
}
```

Run from project root:
```powershell
.\gradlew sonarqube
```

If you cannot run `sonarqube` from Gradle, you can use `sonar-scanner` CLI and configure `sonar-project.properties` with the same properties.

----
## 4) Project structure & important classes

Below are important classes from the project and brief explanation:
- `Report` — a simple container for cost/duration/people; annotated with `@CheckBeforeDelivery` and extends `Procedures`.
- `ProjectProcess` — holds timeline info and evaluates deadline achievability.
- DAO implementations (e.g., `SplunkMonitoringDAOImpl`) — perform DB operations.

### `Report.java`  — concise review
```java
public class Report extends Procedures {
  private float totalCost; // ...
  private int totalDays;
  private int totalPeople;
  public Report(float totalCost, int totalDays, int totalPeople, int timeOffScheduled) {
    super(timeOffScheduled);
    this.totalCost = totalCost;
    this.totalDays = totalDays;
    this.totalPeople = totalPeople;
  }
  // getters, setters, printReport(), equals/hashCode
}
```


----
## 5) Unit testing: example with JUnit + Mockito

We'll create a minimal unit test for `Report` that *does not* try to inject or mock the `static final Logger` field (that caused errors). Instead, test behaviour of getters, setters, equals/hashCode and `printReport()` doesn't throw.

Add these dependencies to your Gradle `build.gradle` (or Maven equivalent):
```groovy
dependencies {
  testImplementation 'org.junit.jupiter:junit-jupiter-api:5.11.0'
  testRuntimeOnly 'org.junit.jupiter:junit-jupiter-engine:5.11.0'
  testImplementation 'org.mockito:mockito-core:5.11.0'
}
test {
  useJUnitPlatform()
}
```

### Example test
```java
package com.solvd.it.company;
import org.junit.jupiter.api.Test;
import static org.junit.jupiter.api.Assertions.*;
class ReportTest {
  @Test
  void gettersSettersAndEqualsHashCode() {
    Report a = new Report(10000f, 30, 5, 2);
    Report b = new Report(10000f, 30, 5, 2);
    assertEquals(10000f, a.getTotalCost());
    assertEquals(30, a.getTotalDays());
    assertEquals(5, a.getTotalPeople());
    assertEquals(a, b);
    assertEquals(a.hashCode(), b.hashCode());
  }
  @Test
  void printReportDoesNotThrow() {
    Report r = new Report(5000f, 15, 3, 1);
    assertDoesNotThrow(() -> r.printReport());
  }
}
```

This avoids mocking `LOGGER` (static final) and focuses on object behaviour which is easy and robust to test.

----
## 6) Troubleshooting: logger mocking & static final fields

- Java `static final` fields cannot be reassigned via reflection on modern JVMs (access restrictions). Instead:
  - Avoid mocking the `Logger` directly; test behaviour rather than log output.
  - If you absolutely must capture logs, configure Log4j to write to an in-memory appender or file and read from it; or use Log4j test utilities.
- For `NoSuchElementException` during tests: ensure any `Scanner` reads are fed via test input or remove interactive prompts for unit tests.
- For `InputMismatchException` or parsing errors: ensure parsing calls use correct formats (e.g., `LocalDate.parse(..., DateTimeFormatter.ofPattern("MM/dd/yyyy"))`).


----
## 7) DB DAO notes (`SplunkMonitoringDAOImpl`)

- Created `save` method correctly with requests generated keys. The DB exception earlier (`Duplicate entry '101' for key ...`) indicates: your test data attempted to insert a record with a primary key that already exists (or `project_code` is unique).
- Fixes:
  1. Ensure the table's primary key is an auto-increment `id` (and you're not inserting it manually), and that `project_code` is not marked `PRIMARY` unless you expect unique codes.
  2. In tests, either use a dedicated test database schema and clear it before tests, or mock the DAO's database interactions with Mockito.

Example mock usage for DAO:
```java
SplunkMonitoringDAO dao = mock(SplunkMonitoringDAO.class);
when(dao.save(any())).thenReturn(123);
int id = dao.save(new SplunkMonitoring(...));
assertEquals(123, id);
```


----
## 8) Useful commands (Windows PowerShell)

Build project:
```powershell
.\gradlew build --refresh-dependencies
.\gradlew test
```
Run SonarQube (if configured):
```powershell
.\gradlew sonarqube
```
Run a single test class:
```powershell
.\gradlew test --tests com.solvd.it.company.ReportTest
```


----
## 9) Full example: simple Mockito test to validate `Report` equals logic

```java
import static org.mockito.Mockito.*;
import org.junit.jupiter.api.Test;
import static org.junit.jupiter.api.Assertions.*;

class ReportMockitoTest {
  @Test
  void exampleMockedReportConstruction() {
    // We don't need to mock Report, but show a simple Mockito usage
    Report r = spy(new Report(1f,1,1,0));
    doReturn(42).when(r).getTotalDays();
    assertEquals(42, r.getTotalDays());
  }
}
```
